# 二级页表的python实现
题干：有一台假想的计算机，页大小（page size）为32 Bytes，支持32KB的虚拟地址空间（virtual address space）,有4KB的物理内存空间（physical memory），采用二级页表，一个页目录项（page directory entry, PDE）大小为1 Byte,一个页表项（page-table entries, PTEs）大小为1 Byte，1个页目录表大小为32 Bytes，1个页表大小为32 Bytes。页目录基址寄存器（page directory base register，PDBR）保存了页目录表的物理地址（按页对齐）其值为0x220（十进制为544）。

PTE格式（8 bit）:
```
VALID | PFN6 ... PFN0
```
PDE格式（8 bit）:
```
VALID | PT6 ... PT0
```
其中：
```
VALID==1表示，表示映射存在；VALID==0表示，表示映射不存在。 PFN6..0:页帧号 PT6..0:页表的物理基址>>5
```

In [16]:
# 内存布局
physical_memory = """page 00: 7f 7f 7f 7f 7f 7f b2 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 01: 01 19 1d 11 05 1a 1e 15 0f 0b 0a 06 14 0d 06 03 14 09 0f 0a 1b 1b 00 06 0a 1e 18 0a 06 14 19 1c 
page 02: ba 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 9f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f a4 7f 7f 7f 7f 
page 03: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 04: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 9c 
page 05: 1e 12 0c 05 0f 1e 17 10 1a 07 0f 1d 11 0e 08 10 1d 00 18 19 1b 16 19 10 11 0d 01 1a 11 06 0f 0f 
page 06: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 07: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 08: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f bd 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 09: 06 17 04 06 05 0b 01 0b 12 15 1a 02 13 06 0a 11 19 14 1e 00 09 0e 01 01 0b 11 04 02 09 16 11 1e 
page 0a: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 0b: 0f 1c 07 09 12 11 11 0c 01 12 08 13 0d 08 1e 09 1e 0e 10 08 05 15 0e 12 05 0f 14 17 14 0c 15 12 
page 0c: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 0d: 09 08 01 00 04 0c 1c 13 07 15 08 0f 09 1d 0a 18 13 1a 12 1a 0c 15 13 10 11 10 0c 1b 13 11 01 0f 
page 0e: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 0f: 7f 7f 7f f1 7f 7f 7f b4 7f 7f ca c2 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 10: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 11: da f7 f2 a8 96 c5 9d 94 c8 b9 7f c4 98 e5 7f 7f d3 a1 82 8f a6 fb bf f0 7f 84 d2 a0 88 80 c9 92 
page 12: 7f 7f dd 7f 7f 7f 7f 7f 7f 7f 7f 95 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 13: 1c 00 04 14 01 0b 04 0e 1a 1c 01 01 1a 01 08 03 02 00 13 17 15 1a 14 0c 13 1e 13 07 01 1c 12 0a 
page 14: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f ad 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 15: 15 11 1c 05 0c 16 01 03 10 08 03 08 13 10 02 02 1a 13 05 1a 00 0b 1a 16 08 1b 12 1a 1b 0d 14 10 
page 16: 7f 7f 7f 7f 7f d9 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 17: 15 09 1c 0d 0e 00 00 03 13 14 1d 0d 15 0a 02 0d 15 18 1d 19 11 11 0f 0e 15 11 1a 0d 0e 19 14 10 
page 18: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f b5 7f 7f fa 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 93 7f 7f 7f 7f 7f 7f 
page 19: 0d 1b 11 11 0c 06 13 02 18 01 17 0b 15 12 04 10 02 07 0b 08 14 1e 0b 19 06 15 0a 10 02 19 06 0e 
page 1a: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 1b: 17 01 15 16 0a 0f 05 16 01 1e 19 13 08 1c 1d 04 18 1a 14 15 0d 1c 1e 0b 18 0d 05 0a 03 1c 1d 03 
page 1c: 07 06 0a 0e 1b 14 13 1a 0b 0d 07 19 1b 1c 12 03 02 18 1b 10 1c 1c 17 0b 0e 0a 0c 0e 00 03 05 11 
page 1d: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f a5 7f 7f 7f 7f 7f 7f 7f 
page 1e: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 1f: 07 02 11 02 02 13 09 18 0f 1a 04 0f 18 02 1d 05 18 1e 19 09 03 0f 1c 09 1b 04 1c 00 09 1b 18 1c 
page 20: 7f 7f 7f e1 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f db 7f 7f 7f 
page 21: 7f 7f 7f 7f 7f 7f 99 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f d5 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 22: 11 1c 0e 08 18 19 00 14 02 03 1b 01 07 1e 0d 03 05 07 17 03 1c 0b 1e 1c 09 07 0e 03 14 01 00 1e 
page 23: 0c 12 1e 15 09 10 0d 0f 0c 05 0c 0c 06 19 0b 04 11 1a 18 13 16 0a 04 07 0a 08 0e 04 04 07 06 18 
page 24: 1e 07 0f 16 15 0d 18 0f 16 0f 1e 16 02 04 1d 07 00 07 0e 1d 0c 16 19 02 12 1e 11 10 0a 1d 1e 03 
page 25: 06 1b 07 11 0e 0a 0c 00 05 13 17 06 05 1c 03 18 1b 04 19 04 01 08 0a 12 0d 18 14 03 06 17 03 1d 
page 26: 7f 7f 7f 7f 7f a7 7f 7f 7f 7f 7f 7f 7f f5 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 97 7f 7f 
page 27: 0f 12 11 1e 05 13 1e 16 11 0b 0c 1d 1b 14 1b 1c 1a 0b 0e 18 0b 17 0e 0a 0e 03 0a 0c 00 05 08 1a 
page 28: 7f 7f 7f e8 7f 7f aa 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 8b 7f 7f 7f 7f 7f 7f 7f 7f 7f f9 7f 7f 7f 7f 
page 29: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 2a: 1e 14 18 0e 0e 0f 17 16 06 0d 10 11 1c 1a 04 0b 03 00 02 05 17 14 18 08 04 1e 19 1d 00 0c 13 16 
page 2b: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 2c: 13 12 0e 00 07 05 12 1a 18 18 19 07 1a 14 02 00 00 17 07 03 13 0a 02 04 10 05 08 16 0e 09 0e 00 
page 2d: 18 1c 09 1b 0a 14 1e 17 01 0d 0b 09 19 0d 15 13 10 01 14 1b 05 13 0f 0a 16 1e 00 1b 0b 19 05 16 
page 2e: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 2f: 0a 17 0c 0b 11 06 02 07 00 14 0e 13 11 01 19 19 00 0d 18 0c 1a 15 14 1c 18 12 01 0e 15 10 12 0a 
page 30: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 31: 0d 0c 1a 12 16 1b 17 0f 17 13 14 13 0c 13 13 1d 18 1a 17 19 12 0a 08 15 1b 10 04 19 0f 0e 01 0f 
page 32: 18 1e 1c 14 0a 07 18 1c 1d 05 12 0f 0d 18 1d 16 15 15 14 10 07 18 03 13 0b 11 13 0e 1e 07 00 1d 
page 33: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 34: 09 07 17 00 05 0d 00 13 12 19 06 08 10 08 12 07 15 18 19 1e 10 0f 1a 17 11 0b 08 03 19 03 17 10 
page 35: 03 1b 0f 06 12 19 00 00 04 16 04 15 15 10 06 1e 18 10 06 14 0c 11 09 13 01 09 1e 1b 09 1a 09 1d 
page 36: 0b 14 00 0f 1d 15 0c 15 12 06 06 1b 08 02 19 10 0b 0f 16 05 14 16 19 08 12 07 11 05 18 1a 0a 06 
page 37: 07 08 10 0e 0c 03 0b 14 10 10 1a 16 15 00 09 15 04 1c 04 1b 06 1a 1a 0a 1b 04 1a 0b 0d 03 12 08 
page 38: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 39: 7f 7f 7f 7f 7f 7f 7f 7f 7f cb 7f 7f 7f 7f cc 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f a3 
page 3a: 1b 07 0d 15 1c 15 13 0b 0d 13 0b 18 00 17 17 09 00 0a 12 18 1a 06 02 03 1e 14 03 15 1d 12 07 1d 
page 3b: 0e 0f 05 10 0d 1b 16 0e 04 04 1e 12 06 19 06 0e 1b 03 03 01 04 0b 09 08 00 0f 0d 16 09 12 09 17 
page 3c: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 3d: 14 1a 00 05 0a 09 19 07 09 0f 1d 09 13 01 0d 1c 06 0b 14 11 11 12 14 0a 0a 0c 10 11 1b 0c 0d 19 
page 3e: 00 07 1b 01 14 0f 1e 1e 03 08 1e 0a 05 1c 13 09 11 0d 0e 11 05 13 1d 12 18 08 04 00 1e 03 0b 14 
page 3f: 7f 7f e7 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f e2 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 40: 0d 15 16 07 0f 0e 06 0c 11 1c 1c 08 14 01 02 0f 1b 04 17 1b 09 15 1a 0b 15 16 12 1a 1b 1d 11 05 
page 41: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 42: 01 0a 0c 06 1b 0d 14 16 17 16 17 07 1e 04 1c 1a 1a 01 02 19 0e 0b 1e 01 10 0d 03 0c 15 1b 00 10 
page 43: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 44: d7 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f ac 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 45: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f be e9 b7 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 46: 11 13 02 05 1d 08 02 08 16 08 06 08 0e 16 09 1b 1e 19 02 09 1c 1a 04 0d 0b 17 00 09 17 1b 01 12 
page 47: 05 1e 1a 03 0a 16 16 1d 0d 19 14 09 12 1b 1a 0f 12 01 07 18 0c 05 11 15 14 0b 0d 0f 18 10 0c 0f 
page 48: 9b 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 49: 7f 7f 7f 7f 7f 7f 7f dc 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f ec 7f fc 7f 7f 7f 
page 4a: 09 10 08 00 09 02 0f 0f 1a 17 17 1e 08 0b 07 03 0f 0f 04 1c 1e 02 00 01 16 1a 02 00 10 0a 00 00 
page 4b: 13 17 07 01 19 09 19 1d 13 03 1a 13 07 06 0f 03 1c 15 19 0b 1c 04 16 07 00 03 06 17 0b 0e 13 08 
page 4c: 09 01 1c 1e 1e 03 06 13 1e 10 15 14 08 10 09 07 02 08 1e 0d 14 13 1d 0c 09 0a 09 1a 1b 09 0a 10 
page 4d: 00 1d 1c 13 0b 11 1b 0e 18 12 0d 1c 0c 12 01 0e 01 15 00 01 03 04 0f 0b 08 1e 1c 14 18 19 07 19 
page 4e: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 4f: 0d 19 0d 19 01 16 03 0c 0d 05 0d 1a 01 06 1e 0d 0c 1c 18 05 12 05 18 11 18 02 1c 07 1a 0d 1b 03 
page 50: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 51: 0d 04 00 04 08 0e 00 02 18 1e 0d 0e 12 03 10 14 1d 13 10 0c 1c 10 0e 16 0d 02 12 1a 0b 02 03 1c 
page 52: 7f 7f 7f 7f 7f 7f 7f c6 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f c7 7f 7f df d1 7f 7f 
page 53: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f af 7f 7f 7f 7f 7f c0 7f 7f 
page 54: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 55: 08 0e 18 1a 18 14 02 0c 14 09 0f 1c 03 1a 03 0b 1c 06 10 0c 1b 1e 08 0f 1b 10 06 17 0a 0f 00 1e 
page 56: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 57: 0f 1e 07 16 02 05 1c 06 17 12 1a 0b 0a 09 1a 12 1d 1b 04 11 03 01 02 1a 18 19 0a 13 18 0b 11 06 
page 58: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 59: 07 0f 0f 1c 02 10 11 04 0b 04 1b 0a 02 0e 10 1b 16 06 1c 00 15 01 19 05 18 19 17 03 0c 03 16 1e 
page 5a: 7f 7f 7f b1 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 85 7f 
page 5b: 04 0e 13 04 02 12 07 13 05 1a 09 0d 11 1b 1c 1a 15 02 05 10 1e 16 05 0b 1d 0f 1a 1a 18 13 14 0a 
page 5c: 19 1e 07 06 17 17 0f 0f 0c 1b 18 12 01 1a 0e 05 09 15 00 03 09 1b 17 1e 10 11 11 10 10 19 1d 0c 
page 5d: 07 16 01 0f 11 15 1c 18 11 0f 00 11 11 17 05 12 01 16 19 0d 15 14 09 02 17 0b 05 0d 19 1d 11 1e 
page 5e: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 5f: 15 05 0d 04 16 14 07 01 1d 19 11 10 1a 0e 0c 0a 07 00 14 0c 11 01 0b 04 03 08 19 0c 0c 12 07 00 
page 60: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 61: 0f 0d 14 18 02 00 19 0d 17 00 0d 16 07 1d 1b 00 00 10 1d 0b 06 0d 00 06 0d 0f 07 07 06 0e 08 00 
page 62: 0f 11 0b 09 0d 10 0e 1a 02 06 1d 12 13 13 07 0c 06 04 1e 17 1b 00 15 09 0f 14 00 0b 11 1b 0f 09 
page 63: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 64: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 65: 7f 7f 7f 7f 7f 7f 7f 7f 7f ff 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 89 7f 7f 7f 7f 
page 66: 02 01 0b 16 09 04 18 19 1a 09 0d 07 11 0a 0a 18 1d 12 03 14 0d 1e 16 19 15 10 1b 19 09 04 0b 10 
page 67: 0a 03 04 07 02 10 15 11 15 07 06 11 1b 0d 00 00 09 13 02 06 15 1e 0a 12 10 00 0a 04 07 17 15 01 
page 68: 09 0a 0a 17 0f 10 04 1c 0a 0a 02 1e 0e 1c 1c 1e 19 1c 1c 18 04 10 11 1e 18 15 17 0b 1d 13 0c 0e 
page 69: 1d 11 13 16 09 12 0e 0f 0c 10 06 07 06 12 07 18 1c 17 0a 1d 0a 0b 12 14 0c 18 1a 08 06 0c 15 14 
page 6a: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 6b: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 6c: 00 1b 13 09 0d 17 18 05 13 05 1e 0d 1c 16 12 08 10 04 04 16 0b 17 07 16 16 09 03 0c 0f 03 05 01 
page 6d: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 6e: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 6f: 0d 1e 06 03 04 12 03 08 06 01 1d 0a 1d 16 06 1c 00 1e 01 07 11 00 17 02 19 01 10 06 08 0f 0b 0c 
page 70: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f cf 7f 7f 7f 7f 7f 7f 7f 7f 81 b6 7f 7f 7f 7f 7f 
page 71: 06 01 15 19 1d 13 0a 19 03 15 02 0c 0f 0b 05 07 19 0e 11 06 16 0a 12 1c 1e 01 18 1a 09 0b 11 1c 
page 72: 7f 7f f3 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 8d 7f 7f 7f a2 7f 7f 7f 7f 7f 7f 7f 7f e6 7f 7f 7f 
page 73: 0e 19 05 04 1c 08 01 0f 1e 19 0c 1e 18 1a 14 0c 1c 0e 1c 11 1c 0e 0d 12 09 04 12 1a 08 1a 18 18 
page 74: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 75: 0f 1a 1e 17 0d 08 03 19 04 11 0e 01 06 19 10 1c 1c 02 15 01 0d 0d 1d 12 05 0f 10 06 0b 1b 1e 18 
page 76: 08 10 0d 17 1a 07 08 15 0c 04 06 11 12 1d 10 12 04 0c 08 15 08 06 0b 0e 0a 12 05 1b 15 10 01 0a 
page 77: 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f bb 7f 7f 7f 7f 7f 7f 7f 7f 
page 78: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 79: 01 08 13 1d 06 07 1a 0a 0c 0b 01 1d 15 00 03 04 0b 1b 04 07 09 0f 1b 10 1c 10 0d 1b 12 0e 0f 0b 
page 7a: 12 18 1a 06 02 12 0b 16 09 0d 19 02 0c 04 10 16 1e 17 04 0d 10 13 15 1e 1d 06 04 1e 04 1e 03 12 
page 7b: 7f f6 7f 7f 7f 7f 7f ef 7f 7f cd 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 7f 
page 7c: 09 0f 0b 10 00 0d 0d 09 0c 18 15 0f 14 0b 06 00 08 12 1b 19 0f 1e 0e 19 0c 17 1e 09 05 13 10 0b 
page 7d: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 7e: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
page 7f: 11 03 11 00 1b 0a 0b 11 13 12 0f 13 1a 0d 0f 19 00 04 0a 06 1b 04 03 09 0f 19 1e 1a 12 01 01 13""" 

# 解题思路
我们的目标是给定一个虚拟地址(int)，返回对应的物理地址(int)，如果不存在映射，返回None，即实现
```python
def translate(virtual_address: int) -> Optional[int]:
```
为此，我们依次实现两次页表查询，第一次查询页目录表，第二次查询页表。最后拼接页表项中的页帧号和页内偏移，得到物理地址。
首先实现第一次查询页目录表:
```python
def get_pde(virtual_address: int) -> Optional[int]:
```
这个函数的实现逻辑是：
1. 一个目录页存储了32个页目录项，因此先取出虚拟地址的高5 bit(我们使用的是15 bit的虚拟地址)，得到页目录表的索引
2. 计算页目录项的物理地址 = PDBR + 页目录表索引 * 目录项大小(1 Byte) （不难看出我们其实只有一页目录表，因为32（一页的目录项数）* 32（一个目录项对应的的页表项数）* 32（页内偏移）= 32KB，已经吃满地址空间）
3. 访问物理内存，取出对应的页目录项
4. 得到页目录项后，检查VALID位，如果为0，返回None，否则返回页表的物理基址

然后实现第二次查询页表:
```python
def get_pte(page_table_base: int, virtual_address: int) -> Optional[int]:
```
这个函数的实现逻辑是：
1. 先取出虚拟地址的中间5 bit，得到页表的索引
2. 计算页表的物理地址 = 页表基址 + 页表索引 * 页表项大小(1 Byte)
3. 访问物理内存，取出对应的页表项
4. 得到页表项后，检查VALID位，如果为0，返回None，否则返回页帧号

最后进行拼接即可。

我还实现了一个辅助函数，用于访问物理内存:
```python
def read_physical_memory(physical_address: int) -> int:
```
这个函数的实现逻辑是：
1. 计算页号 = 物理地址 >> 5
2. 计算页内偏移 = 物理地址 & 0x1F
3. 访问物理内存，取出对应的字节
4. 返回该字节的整数值，如果访问越界，抛出异常

In [17]:
import re
from typing import Optional


def read_physical_memory(physical_address: int) -> int:
    """
    Read a byte from the given physical memory address.
    """
    global physical_memory
    page_number = (physical_address >> 5) & 0x7f # (高7位)
    offset = physical_address & 0x1f # (低5位)
    # 正则匹配找到对应页
    match = re.search(rf'page {page_number:02x}:\s*([0-9a-f]{{2}}(?:\s+[0-9a-f]{{2}}){{31}})', physical_memory, re.IGNORECASE)
    if not match:
        raise ValueError(f"Page {page_number:02x} not found in physical memory.")
    page_data = match.group(1).strip().split() # 分割成字节列表，每个字节是一个元素，对应16进制字符串
    return int(page_data[offset], 16) # 返回对应偏移的字节，转换为整数


def get_pde(virtual_address: int) -> Optional[int]:
    """
    Get the Page Directory Entry (PDE) for the given virtual address.

    1. 一个目录页存储了32个页目录项，因此先取出虚拟地址的高5 bit(我们使用的是15 bit的虚拟地址)，得到页目录表的索引
    2. 计算页目录项的物理地址 = PDBR + 页目录表索引 * 目录项大小(1 Byte) （不难看出我们其实只有一页目录表，因为32（一页的目录项数）* 32（一个目录项对应的的页表项数）* 32（页内偏移）= 32KB，已经吃满地址空间）
    3. 访问物理内存，取出对应的页目录项
    4. 得到页目录项后，检查VALID位，如果为0，返回None，否则返回页表的物理基址
    """
    PDBR = 0x220 # 页目录基址
    page_directory_index = (virtual_address >> 10) & 0x1f # 虚拟地址的高5 bit
    pde_physical_address = PDBR + page_directory_index # 页目录项的物理地址
    pde = read_physical_memory(pde_physical_address) # 读取页目录项
    if (pde & 0x80) == 0: # 检查VALID位（最高位）
        return None
    return pde & 0x7f # 返回页表的物理基址（低7位）


def get_pte(page_table_base: int, virtual_address: int) -> Optional[int]:
    """
    Get the Page Table Entry (PTE) for the given virtual address.

    1. 先取出虚拟地址的中间5 bit，得到页表的索引
    2. 计算页表的物理地址 = 页表基址 + 页表索引 * 页表项大小(1 Byte)
    3. 访问物理内存，取出对应的页表项
    4. 得到页表项后，检查VALID位，如果为0，返回None，否则返回页帧号
    """
    page_table_index = (virtual_address >> 5) & 0x1f # 虚拟地址的中间5 bit
    pte_physical_address = (page_table_base << 5) + page_table_index # 页表项的物理地址，注意基址左移5位作为高7位
    pte = read_physical_memory(pte_physical_address) # 读取页表项
    if (pte & 0x80) == 0: # 检查VALID位（最高位）
        return None
    return pte & 0x7f # 返回页帧号（低7位）


def translate(virtual_address: int) -> Optional[str]:
    """
    Translate the given virtual address to a physical address.
    """
    pde = get_pde(virtual_address)
    pde_index = (virtual_address >> 10) & 0x1f
    pde_valid = 1 if pde is not None else 0
    pde_pfn = pde if pde is not None else 0x7f

    pte = None
    pte_index = (virtual_address >> 5) & 0x1f
    if pde is not None:
        pte = get_pte(pde, virtual_address)
    pte_valid = 1 if pte is not None else 0
    pte_pfn = pte if pte is not None else 0x7f

    # Fault on invalid PDE
    if pde is None:
        return (f"Virtual Address {virtual_address:04x}:  --> pde index:0x{pde_index:x}  "
                f"pde contents:(valid {pde_valid}, pfn 0x{pde_pfn:x})      "
                f"--> Fault (page directory entry not valid)")

    # Fault on invalid PTE
    if pte is None:
        return (f"Virtual Address {virtual_address:04x}:  --> pde index:0x{pde_index:x}  "
                f"pde contents:(valid {pde_valid}, pfn 0x{pde_pfn:x})    "
                f"--> pte index:0x{pte_index:x}  pte contents:(valid {pte_valid}, pfn 0x{pte_pfn:x})      "
                f"--> Fault (page table entry not valid)")

    # Successful translation
    page_offset = virtual_address & 0x1f
    physical_address = (pte << 5) | page_offset
    value = read_physical_memory(physical_address)
    return (f"Virtual Address {virtual_address:04x}:  --> pde index:0x{pde_index:x}  "
            f"pde contents:(valid {pde_valid}, pfn 0x{pde_pfn:x})    "
            f"--> pte index:0x{pte_index:x}  pte contents:(valid {pte_valid}, pfn 0x{pte_pfn:x})      "
            f"--> Translates to Physical Address {hex(physical_address)} --> Value: {value}")


if __name__ == "__main__":
    test_virtual_addresses = [
        0x6c74, 0x6b22, 0x03df, 0x69dc, 0x317a, 0x4546, 0x2c03, 0x7fd7, 0x390e, 0x748b
    ]
    for va in test_virtual_addresses:
        result = translate(va)
        print(result)

Virtual Address 6c74:  --> pde index:0x1b  pde contents:(valid 1, pfn 0x20)    --> pte index:0x3  pte contents:(valid 1, pfn 0x61)      --> Translates to Physical Address 0xc34 --> Value: 6
Virtual Address 6b22:  --> pde index:0x1a  pde contents:(valid 1, pfn 0x52)    --> pte index:0x19  pte contents:(valid 1, pfn 0x47)      --> Translates to Physical Address 0x8e2 --> Value: 26
Virtual Address 03df:  --> pde index:0x0  pde contents:(valid 1, pfn 0x5a)    --> pte index:0x1e  pte contents:(valid 1, pfn 0x5)      --> Translates to Physical Address 0xbf --> Value: 15
Virtual Address 69dc:  --> pde index:0x1a  pde contents:(valid 1, pfn 0x52)    --> pte index:0xe  pte contents:(valid 0, pfn 0x7f)      --> Fault (page table entry not valid)
Virtual Address 317a:  --> pde index:0xc  pde contents:(valid 1, pfn 0x18)    --> pte index:0xb  pte contents:(valid 1, pfn 0x35)      --> Translates to Physical Address 0x6ba --> Value: 30
Virtual Address 4546:  --> pde index:0x11  pde contents:(valid 1